<a href="https://colab.research.google.com/github/saerarawas/AAI_635O_B11_202520-Recommender-System/blob/main/Week3/Saera__Rawas_Recommender_System_Course_Project_GitHub_May_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Graded Assessment -- AAI 6350 Recommender Systems Course --

# Part 1: Recommendation System Using GCNN [weight: 40\%]

# Step 1: Data Preparation
- Load the Data: Read the Excel file and extract the relevant columns (CustomerID, StockCode, Quantity).
- Data Cleaning: Ensure there are no missing values in the relevant columns.
- Create Interaction Matrix: Construct an adjacency matrix where rows represent customers and columns represent items. The values in the matrix will be the quantities purchased.

In [1]:
import pandas as pd
import numpy as np

# Load the dataset
data = pd.read_excel("/content/Rec_sys_data.xlsx")

# Create a pivot table to form the interaction matrix
interaction_matrix = data.pivot_table(index='CustomerID', columns='StockCode', values='Quantity', fill_value=0)

# Convert to a NumPy array for further processing
interaction_matrix = interaction_matrix.values

# Step 2: Graph Construction [25 points]
- Graph Representation: Each customer and item will be a node in the graph. An edge exists between a customer and an item if the customer has purchased that item.
- Adjacency Matrix: Create an adjacency matrix where the rows represent customers and the columns represent items.

In [2]:
# Get unique customers and items (StockCode)
customers = data['CustomerID'].unique()
items = data['StockCode'].unique()

# Create mappings from customer/item IDs to matrix indices
customer_to_index = {customer: i for i, customer in enumerate(customers)}
item_to_index = {item: j for j, item in enumerate(items)}

# Initialize the adjacency matrix with zeros
num_customers = len(customers)
num_items = len(items)
adjacency_matrix = np.zeros((num_customers, num_items), dtype=int)

# Populate the adjacency matrix: 1 if a customer purchased an item, 0 otherwise
for index, row in data.iterrows():
    customer_id = row['CustomerID']
    item_id = row['StockCode']
    if customer_id in customer_to_index and item_id in item_to_index:
        customer_index = customer_to_index[customer_id]
        item_index = item_to_index[item_id]
        adjacency_matrix[customer_index, item_index] = 1

# Print the shape of the adjacency matrix and a few rows to verify
print("Shape of the Adjacency Matrix (Customers x Items):", adjacency_matrix.shape)
print("\nFirst 5 rows of the Adjacency Matrix:")
print(adjacency_matrix[:5])

# Create a DataFrame for better visualization if needed
adjacency_df = pd.DataFrame(adjacency_matrix, index=customers, columns=items)
print("\nAdjacency DataFrame (first 5 rows and columns):")
print(adjacency_df.iloc[:5, :5])

Shape of the Adjacency Matrix (Customers x Items): (3647, 3538)

First 5 rows of the Adjacency Matrix:
[[1 1 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]

Adjacency DataFrame (first 5 rows and columns):
       84029E  71053  21730  84406B  22752
17850       1      1      1       1      1
13047       0      0      0       0      0
12583       0      0      0       0      0
13748       0      0      0       0      0
15100       0      0      0       0      0


# Step 3: Model Definition (GCNN) [35 points]
- Define the GCNN Architecture: Use a library like PyTorch Geometric or TensorFlow with Keras to define the GCNN model.
- The model will consist of graph convolutional layers that learn representations for both customers and items.
- Prepare Data for Training: Convert the adjacency matrix and features into a format suitable for the GCNN.

In [3]:
import torch

!pip uninstall torch-scatter torch-sparse torch-geometric torch-cluster  --y
!pip install torch-scatter -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install torch-cluster -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install git+https://github.com/pyg-team/pytorch_geometric.git

Looking in links: https://data.pyg.org/whl/torch-2.6.0+cu124.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 86.7 MB/s eta 0:00:00
Looking in links: https://data.pyg.org/whl/torch-2.6.0+cu124.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 39.8 MB/s eta 0:00:00
Looking in links: https://data.pyg.org/whl/torch-2.6.0+cu124.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 31.6 MB/s eta 0:00:00
  Cloning https://github.com/pyg-team/pytorch_geometric.git to /tmp/pip-req-build-frdon72v
  Running command git clone --filter=blob:none --quiet https://github.com/pyg-team/pytorch_geometric.git /tmp/pip-req-build-frdon72v
  Resolved https://github.com/pyg-team/pytorch_geometric.git to commit 433b3c96e2494923124caa0791a966ce25d14257
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 3.6 MB/s eta 0:00:00
  Created wh

In [4]:
import torch
from torch_geometric.data import Data

# Convert adjacency matrix to edge list (source and target)
edges = []

for customer_index in range(num_customers):
    for item_index in range(num_items):
        if adjacency_matrix[customer_index, item_index] == 1:
            edges.append([customer_index, num_customers + item_index])  # Customer nodes: 0 to num_customers-1, Item nodes: num_customers to num_customers+num_items-1

edges = torch.tensor(edges, dtype=torch.long).t().contiguous()


# If you don't have additional features, you can just use ones as dummy features
customer_features = torch.ones(num_customers, 1)  # Example: all customers having the same feature
item_features = torch.ones(num_items, 1)  # Example: all items having the same feature
node_features = torch.cat([customer_features, item_features], dim=0)  # Concatenate customer and item features

# Create the Data object (PyTorch Geometric format)
data = Data(x=node_features, edge_index=edges)


In [5]:
import torch
import torch.nn as nn
import torch_geometric.nn as pyg_nn

class GCNN(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GCNN, self).__init__()
        self.conv1 = pyg_nn.GCNConv(in_channels, hidden_channels)
        self.conv2 = pyg_nn.GCNConv(hidden_channels, out_channels)
        self.fc = nn.Linear(out_channels, 1)  # Output a single prediction (e.g., purchase likelihood)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        # Apply first convolutional layer
        x = self.conv1(x, edge_index)
        x = torch.relu(x)

        # Apply second convolutional layer
        x = self.conv2(x, edge_index)
        x = torch.relu(x)

        # Apply a fully connected layer (optional)
        x = self.fc(x)

        return x


In [6]:
from torch.optim import Adam

# Set model, optimizer, and loss function
model = GCNN(in_channels=1, hidden_channels=64, out_channels=32)  # Example channel sizes
optimizer = Adam(model.parameters(), lr=0.01)
criterion = nn.BCEWithLogitsLoss()  # Use BCEWithLogitsLoss for binary classification (purchase/no-purchase)

# Example training loop
epochs = 100
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()

    # Forward pass
    out = model(data)

    # Get edge indices
    edge_index = data.edge_index

    # Create target tensor based on edges
    target = torch.tensor(adjacency_matrix[edge_index[0], edge_index[1] - num_customers], dtype=torch.float)

    # Compute the loss
    loss = criterion(out[edge_index[0]].view(-1), target.view(-1)) # Select predictions for edges only

    # Backpropagate
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')


Epoch 0, Loss: 0.6499851942062378
Epoch 10, Loss: 0.05860096588730812
Epoch 20, Loss: 0.0002991936053149402
Epoch 30, Loss: 9.804309229366481e-06
Epoch 40, Loss: 2.0146237602602923e-06
Epoch 50, Loss: 1.0453805998622556e-06
Epoch 60, Loss: 8.070801982285047e-07
Epoch 70, Loss: 7.305415579139662e-07
Epoch 80, Loss: 7.032174380583456e-07
Epoch 90, Loss: 6.928557354513032e-07


In [7]:
model.eval()
with torch.no_grad():
    predictions = model(data)
    predictions = torch.sigmoid(predictions).view(-1)  # Convert logits to probabilities


# Step 4: Training the Model [40 points]

- Loss Function: Use a suitable loss function, such as Mean Squared Error (MSE) as we are working with continuous interaction scores.
- Optimizer: Choose an optimizer like Adam or SGD.
- Training Loop: Implement the training loop to update the model weights based on the loss. In each epoch, calculate the predictions using the model, compute the loss between predicted and actual values, and perform backpropagation to update the model's weights.
- Also compute the validation loss to evaluate the model's performance on unseen data, and use early stopping to halt training when the validation loss stops improving, preventing overfitting.

In [8]:
import torch
import torch.nn as nn
from torch.optim import Adam
from sklearn.model_selection import train_test_split
from torch_geometric.data import Data

# 1. Prepare the data with proper train/val split
# Convert edge indices and features to numpy first
edge_index_np = data.edge_index.numpy().T
edge_labels = torch.tensor(adjacency_matrix[edge_index_np[:, 0], edge_index_np[:, 1] - num_customers], dtype=torch.float)

# Split edges into train and validation sets
train_idx, val_idx = train_test_split(
    np.arange(edge_index_np.shape[0]),
    test_size=0.2,
    random_state=42
)

train_edge_index = torch.tensor(edge_index_np[train_idx], dtype=torch.long).t().contiguous()
val_edge_index = torch.tensor(edge_index_np[val_idx], dtype=torch.long).t().contiguous()

train_labels = edge_labels[train_idx]
val_labels = edge_labels[val_idx]

# 2. Modify the model for regression (since you mentioned continuous scores)
class GCNN(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GCNN, self).__init__()
        self.conv1 = pyg_nn.GCNConv(in_channels, hidden_channels)
        self.conv2 = pyg_nn.GCNConv(hidden_channels, out_channels)
        self.fc = nn.Linear(out_channels, 1)

    def forward(self, data, edge_index=None):
        x, full_edge_index = data.x, data.edge_index
        edge_index = full_edge_index if edge_index is None else edge_index

        x = self.conv1(x, full_edge_index)
        x = torch.relu(x)
        x = self.conv2(x, full_edge_index)
        x = torch.relu(x)
        return self.fc(x)

# 3. Initialize model, optimizer, and loss
model = GCNN(in_channels=1, hidden_channels=64, out_channels=32)
optimizer = Adam(model.parameters(), lr=0.01)
criterion = nn.MSELoss()  # For continuous interaction scores

# 4. Training loop with early stopping
best_val_loss = float('inf')
patience = 5
patience_counter = 0

for epoch in range(100):
    model.train()
    optimizer.zero_grad()

    # Forward pass on training edges
preds = model(data, train_edge_index)

# Get predictions only for the relevant edges
train_preds = preds[train_edge_index[0]].view(-1) # Select predictions for edges only

# Calculate the loss using only the predictions for the edges in the training dataset
train_loss = criterion(train_preds, train_labels)
# Backpropagation
train_loss.backward()
optimizer.step()
# Validation
model.eval()
with torch.no_grad():
    val_preds = model(data, val_edge_index) # Get predictions only for validation edges
    val_loss = criterion(val_preds[val_edge_index[0]].view(-1), val_labels) # Select predictions for edges only and compare with labels
# Early stopping check
if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
else:
        patience_counter += 1
import torch
import torch.nn as nn
from torch.optim import Adam
from sklearn.model_selection import train_test_split
from torch_geometric.data import Data
import numpy as np
import torch_geometric.nn as pyg_nn

# 1. Prepare the data with proper train/val split
# Convert edge indices and features to numpy first
edge_index_np = data.edge_index.numpy().T
edge_labels = torch.tensor(adjacency_matrix[edge_index_np[:, 0], edge_index_np[:, 1] - num_customers], dtype=torch.float)

# Split edges into train and validation sets
train_idx, val_idx = train_test_split(
    np.arange(edge_index_np.shape[0]),
    test_size=0.2,
    random_state=42
)

train_edge_index = torch.tensor(edge_index_np[train_idx], dtype=torch.long).t().contiguous()
val_edge_index = torch.tensor(edge_index_np[val_idx], dtype=torch.long).t().contiguous()

train_labels = edge_labels[train_idx]
val_labels = edge_labels[val_idx]

# 2. Modify the model for regression (since you mentioned continuous scores)
class GCNN(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GCNN, self).__init__()
        self.conv1 = pyg_nn.GCNConv(in_channels, hidden_channels)
        self.conv2 = pyg_nn.GCNConv(hidden_channels, out_channels)
        self.fc = nn.Linear(out_channels, 1)

    def forward(self, data, edge_index=None):
        x, full_edge_index = data.x, data.edge_index
        edge_index = full_edge_index if edge_index is None else edge_index

        x = self.conv1(x, full_edge_index)
        x = torch.relu(x)
        x = self.conv2(x, full_edge_index)
        x = torch.relu(x)
        return self.fc(x)

# 3. Initialize model, optimizer, and loss
model = GCNN(in_channels=1, hidden_channels=64, out_channels=32)
optimizer = Adam(model.parameters(), lr=0.01)
criterion = nn.MSELoss()  # For continuous interaction scores

# 4. Training loop with early stopping
best_val_loss = float('inf')
patience = 5
patience_counter = 0

for epoch in range(100):
    model.train()
    optimizer.zero_grad()

    # Forward pass on training edges
    preds = model(data, train_edge_index)

    # Get predictions only for the relevant edges
    train_preds = preds[train_edge_index[0]].view(-1) # Select predictions for edges only

    # Calculate the loss using only the predictions for the edges in the training dataset
    train_loss = criterion(train_preds, train_labels)
    # Backpropagation
    train_loss.backward()
    optimizer.step()
    # Validation
    model.eval()
    with torch.no_grad():
        val_preds = model(data, val_edge_index) # Get predictions only for validation edges
        val_loss = criterion(val_preds[val_edge_index[0]].view(-1), val_labels) # Select predictions for edges only and compare with labels
    # Early stopping check
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print(f"Early stopping at epoch {epoch}")
            break

    if epoch % 5 == 0:
        print(f'Epoch {epoch}: Train Loss: {train_loss.item():.4f}, Val Loss: {val_loss.item():.4f}')


Epoch 0: Train Loss: 0.9092, Val Loss: 0.6144
Epoch 5: Train Loss: 0.0398, Val Loss: 0.0005
Early stopping at epoch 10


# Part 2: Recommendation System Evaluation and Comparison Using GCNN and NeuMF Models [weight: 30\%]

# Step 1: Evaluation [40 points]

To calculate the average precision, recall, and F1 score for all customers, follow these steps:

- Obtain Model Predictions: Use the trained model to predict interaction scores for all customer-item pairs in the validation set.

- Rank Items by Predicted Scores: For each customer, rank items based on the predicted interaction scores in descending order.

- Define Relevant Items: Set a threshold to determine which items are considered relevant (e.g., based on the top-k predictions or actual interactions greater than zero).

- Calculate Precision, Recall, and F1 Score for Each Customer: For each customer, calculate precision, recall, and F1 score using the relevant predicted and actual items.

- Compute Average Precision, Recall, and F1 Score: Calculate the mean of precision, recall, and F1 scores across all customers.

In [9]:
model.eval()
with torch.no_grad():
    output = model(data).view(-1)  # Get raw scores


In [10]:
from collections import defaultdict

# Dictionaries to store predictions and ground truth per customer
predictions_per_customer = defaultdict(list)
actuals_per_customer = defaultdict(set)

# Use val_edge_index instead of val_edges
for i in range(val_edge_index.shape[1]):
    customer_idx = val_edge_index[0, i].item()
    item_idx = val_edge_index[1, i].item() - num_customers  # Convert to item index

    score = output[val_edge_index[0, i]].item()  # Predicted score
    actual = adjacency_matrix[customer_idx, item_idx]

    predictions_per_customer[customer_idx].append((item_idx, score))
    if actual > 0:
        actuals_per_customer[customer_idx].add(item_idx)


In [11]:
from sklearn.metrics import precision_score, recall_score, f1_score

def precision_recall_f1(y_true, y_pred):
    if not y_pred:
        return 0.0, 0.0, 0.0
    y_true_bin = [1 if item in y_true else 0 for item in y_pred]
    y_pred_bin = [1] * len(y_pred)

    tp = sum(y_true_bin)
    precision = tp / len(y_pred)
    recall = tp / len(y_true) if y_true else 0.0
    f1 = (2 * precision * recall) / (precision + recall + 1e-8)  # Avoid divide by zero
    return precision, recall, f1

# Store all scores
precision_list = []
recall_list = []
f1_list = []

k = 10  # Top-k items to recommend

for customer, predictions in predictions_per_customer.items():
    predictions_sorted = sorted(predictions, key=lambda x: x[1], reverse=True)
    top_k_items = [item for item, score in predictions_sorted[:k]]
    actual_items = actuals_per_customer[customer]

    p, r, f = precision_recall_f1(actual_items, top_k_items)
    precision_list.append(p)
    recall_list.append(r)
    f1_list.append(f)

# Compute averages
avg_precision = np.mean(precision_list)
avg_recall = np.mean(recall_list)
avg_f1 = np.mean(f1_list)

print(f"Average Precision@{k}: {avg_precision:.4f}")
print(f"Average Recall@{k}: {avg_recall:.4f}")
print(f"Average F1 Score@{k}: {avg_f1:.4f}")


Average Precision@10: 1.0000
Average Recall@10: 0.8408
Average F1 Score@10: 0.8872


In [12]:
import torch
import torch.nn as nn
import torch_geometric.nn as pyg_nn

class GCNN(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GCNN, self).__init__()
        self.conv1 = pyg_nn.GCNConv(in_channels, hidden_channels)
        self.conv2 = pyg_nn.GCNConv(hidden_channels, out_channels)
        self.fc = nn.Linear(out_channels, 1)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = torch.relu(self.conv1(x, edge_index))
        x = torch.relu(self.conv2(x, edge_index))
        x = self.fc(x)
        return x


In [13]:
from collections import defaultdict
import numpy as np

def evaluate_gcnn(model, data, adjacency_matrix, num_customers, num_items, top_k=10):
    model.eval()
    with torch.no_grad():
        scores = model(data).view(-1)

    item_scores = scores[num_customers:]  # Item nodes only

    precision_list, recall_list, f1_list = [], [], []

    for cust_idx in range(num_customers):
        actual_indices = np.where(adjacency_matrix[cust_idx] > 0)[0]
        if len(actual_indices) == 0:
            continue  # Skip customers with no purchases

        predicted_scores = item_scores  # Same for all customers
        top_k_items = torch.topk(predicted_scores, k=top_k).indices.tolist()

        actual_set = set(actual_indices)
        pred_set = set(top_k_items)

        tp = len(actual_set & pred_set)
        precision = tp / top_k
        recall = tp / len(actual_set)
        f1 = (2 * precision * recall) / (precision + recall + 1e-8)

        precision_list.append(precision)
        recall_list.append(recall)
        f1_list.append(f1)

    return np.mean(precision_list), np.mean(recall_list), np.mean(f1_list)


In [14]:
class NeuMF(nn.Module):
    def __init__(self, num_users, num_items, embedding_dim=32):
        super(NeuMF, self).__init__()
        self.user_embed = nn.Embedding(num_users, embedding_dim)
        self.item_embed = nn.Embedding(num_items, embedding_dim)

        self.mlp = nn.Sequential(
            nn.Linear(2 * embedding_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )

    def forward(self, user_indices, item_indices):
        u = self.user_embed(user_indices)
        v = self.item_embed(item_indices)
        x = torch.cat([u, v], dim=-1)
        return self.mlp(x).squeeze()


In [15]:
def evaluate_neumf(model, adjacency_matrix, top_k=10):
    model.eval()
    precision_list, recall_list, f1_list = [], [], []

    num_users, num_items = adjacency_matrix.shape

    with torch.no_grad():
        for user in range(num_users):
            actual_items = np.where(adjacency_matrix[user] > 0)[0]
            if len(actual_items) == 0:
                continue

            item_indices = torch.arange(num_items)
            user_tensor = torch.full((num_items,), user, dtype=torch.long)

            scores = model(user_tensor, item_indices)
            top_items = torch.topk(scores, k=top_k).indices.tolist()

            actual_set = set(actual_items)
            pred_set = set(top_items)

            tp = len(actual_set & pred_set)
            precision = tp / top_k
            recall = tp / len(actual_set)
            f1 = (2 * precision * recall) / (precision + recall + 1e-8)

            precision_list.append(precision)
            recall_list.append(recall)
            f1_list.append(f1)

    return np.mean(precision_list), np.mean(recall_list), np.mean(f1_list)


In [16]:
gcnn_model = GCNN(in_channels=1, hidden_channels=64, out_channels=32) # Example channel sizes
gcnn_data = data

# Example usage after training both models:
gcnn_p, gcnn_r, gcnn_f1 = evaluate_gcnn(gcnn_model, gcnn_data, adjacency_matrix, num_customers, num_items)
print(f"GCNN -> Precision@10: {gcnn_p:.4f}, Recall@10: {gcnn_r:.4f}, F1: {gcnn_f1:.4f}")

# Make sure num_customers and num_items are defined
neumf_model = NeuMF(num_users=num_customers, num_items=num_items)

neumf_p, neumf_r, neumf_f1 = evaluate_neumf(neumf_model, adjacency_matrix)
print(f"NeuMF -> Precision@10: {neumf_p:.4f}, Recall@10: {neumf_r:.4f}, F1: {neumf_f1:.4f}")

GCNN -> Precision@10: 0.1580, Recall@10: 0.0420, F1: 0.0528
NeuMF -> Precision@10: 0.0155, Recall@10: 0.0036, F1: 0.0046


Evaluation Metrics Comparison

Metric	          GCNN	       NeuMF

Precision@10	    0.1580	    0.0155

Recall@10	        0.0420	    0.0036

F1 Score	        0.0528	    0.0046


Analysis

✅ Precision@10
GCNN achieves a precision of 15.8%, which is 10x higher than NeuMF’s 1.55%.

This means that among the top 10 recommended items, GCNN returns significantly more relevant items than NeuMF.

Implication: GCNN is better at ranking relevant items higher, which is critical in top-K recommendation tasks.

✅ Recall@10
GCNN recall is 4.20%, compared to NeuMF’s 0.36%, indicating it captures more of the relevant items from the full ground truth.

While both recall values are relatively low (typical in sparse recommendation settings), GCNN still retrieves more useful recommendations.

Implication: GCNN explores and captures a broader set of relevant items than NeuMF.

✅ F1 Score
F1 score balances precision and recall; GCNN's score of 0.0528 is far superior to NeuMF’s 0.0046.

This suggests that GCNN maintains a better trade-off between precision and recall.

Why GCNN Outperforms NeuMF
Graph Structural Awareness:

GCNN leverages graph connectivity (e.g., user-item interaction graphs), capturing higher-order relationships, such as indirect connections and user similarity patterns.

NeuMF, being a standard feedforward architecture, lacks this ability to model rich collaborative signals.

Message Passing:

GCNN applies message passing and neighborhood aggregation, allowing the model to incorporate contextual information from nearby nodes (e.g., similar users or items), leading to more accurate recommendations.

Expressiveness:

While NeuMF can model non-linear interactions, it still works on latent factors alone. GCNN enriches these latent representations by integrating topological information, enhancing its expressiveness.

Conclusion
The GCNN model is clearly superior to the NeuMF model across all key metrics. The strength of GCNN lies in its ability to incorporate graph-based relationships, which are crucial in recommendation systems where user-item interactions form sparse and complex networks. Unless NeuMF is significantly enhanced or tailored with more sophisticated architectures, GCNN should be the preferred model in this context.

# Step 2: Generating Recommendations and Evaluating for a Specific Customer [40 points]

1- Mapping Customer IDs to Indices.

2- Get Predicted Scores for the Customer.

3- Rank Items by Predicted Scores.

4- Map Recommended Items to Stock Codes.

5- Compare Recommendations with Actual Interactions.

6- Calculate Precision, Recall, and F1 Score.

In [17]:
customer_to_index = {customer: i for i, customer in enumerate(customers)}
index_to_customer = {i: customer for i, customer in enumerate(customers)}
item_to_index = {item: j for j, item in enumerate(items)}
index_to_item = {j: item for j, item in enumerate(items)}


In [18]:
customer_id = customers[0]  # Or any ID of interest
customer_idx = customer_to_index[customer_id]

model.eval()
with torch.no_grad():
    scores = model(data).view(-1)

# Get predicted scores for all item nodes (offset by num_customers)
item_scores = scores[num_customers:]  # Only item nodes


In [19]:
# Sort items by descending predicted scores
top_k = 10
ranked_items = torch.topk(item_scores, k=top_k).indices.tolist()


In [20]:
recommended_stock_codes = [index_to_item[i] for i in ranked_items]
print(f"Top-{top_k} recommended items for customer {customer_id}:")
print(recommended_stock_codes)


Top-10 recommended items for customer 17850:
[22423, '85123A', 47566, 21212, 22720, 84879, '85099B', 22960, 23298, 22457]


In [21]:
# Get actual items purchased by the customer
actual_purchased_item_indices = np.where(adjacency_matrix[customer_idx] > 0)[0]
actual_stock_codes = [index_to_item[i] for i in actual_purchased_item_indices]

print(f"\nActual purchased items by customer {customer_id}:")
print(actual_stock_codes)



Actual purchased items by customer 17850:
['84029E', 71053, 21730, '84406B', 22752, '85123A', '84029G', 22633, 22632, 20679, 21068, 21871, 82483, 21071, 82486, 37370, '82494L', 82482, '15056BL', 22411, 22803]


In [22]:
def precision_recall_f1_set(y_true, y_pred):
    y_true_set = set(y_true)
    y_pred_set = set(y_pred)
    tp = len(y_true_set & y_pred_set)

    precision = tp / len(y_pred) if y_pred else 0.0
    recall = tp / len(y_true) if y_true else 0.0
    f1 = (2 * precision * recall) / (precision + recall + 1e-8)
    return precision, recall, f1

precision, recall, f1 = precision_recall_f1_set(actual_stock_codes, recommended_stock_codes)

print(f"\nPrecision@{top_k}: {precision:.4f}")
print(f"Recall@{top_k}: {recall:.4f}")
print(f"F1 Score@{top_k}: {f1:.4f}")



Precision@10: 0.1000
Recall@10: 0.0476
F1 Score@10: 0.0645


In [23]:
def compare_gcnn_neumf(customer_id, gcnn_model, neumf_model, data, adjacency_matrix, customer_to_index, items, top_k=10):
    customer_idx = customer_to_index[customer_id]
    num_items = len(items)

    # ===== GCNN Predictions =====
    gcnn_model.eval()
    with torch.no_grad():
        gcnn_scores = gcnn_model(data).view(-1)
        gcnn_item_scores = gcnn_scores[len(customer_to_index):]  # Only item node predictions
        gcnn_top_k_items = torch.topk(gcnn_item_scores, k=top_k).indices.tolist()
        gcnn_stockcodes = [items[i] for i in gcnn_top_k_items]

    # ===== NeuMF Predictions =====
    neumf_model.eval()
    with torch.no_grad():
        item_indices = torch.arange(num_items)
        user_tensor = torch.full((num_items,), customer_idx, dtype=torch.long)
        neumf_scores = neumf_model(user_tensor, item_indices)
        neumf_top_k_items = torch.topk(neumf_scores, k=top_k).indices.tolist()
        neumf_stockcodes = [items[i] for i in neumf_top_k_items]

    # ===== Actual Interactions =====
    actual_indices = np.where(adjacency_matrix[customer_idx] > 0)[0]
    actual_stockcodes = [items[i] for i in actual_indices]

    # ===== Print Results =====
    print(f"\n🧾 Recommendations for Customer ID {customer_id}")
    print(f"GCNN  Top-{top_k} StockCodes:  {gcnn_stockcodes}")
    print(f"NeuMF Top-{top_k} StockCodes:  {neumf_stockcodes}")
    print(f"Actual StockCodes (Purchased): {actual_stockcodes}")

    # Optional: Calculate overlap
    overlap = set(gcnn_stockcodes) & set(neumf_stockcodes)
    print(f"Overlap between GCNN and NeuMF: {overlap if overlap else 'None'}")

# 🔍 Call the function for customer 17850
compare_gcnn_neumf(
    customer_id=17850,
    gcnn_model=gcnn_model,
    neumf_model=neumf_model,
    data=data,
    adjacency_matrix=adjacency_matrix,
    customer_to_index=customer_to_index,
    items=items,
    top_k=10
)



🧾 Recommendations for Customer ID 17850
GCNN  Top-10 StockCodes:  [22423, '85123A', 47566, 21212, 22720, 84879, '85099B', 22960, 23298, 22457]
NeuMF Top-10 StockCodes:  [85125, 37495, 23284, 23503, 21470, 22779, '35911B', 23418, 21220, '79063C']
Actual StockCodes (Purchased): ['84029E', 71053, 21730, '84406B', 22752, '85123A', '84029G', 22633, 22632, 20679, 21068, 21871, 82483, 21071, 82486, 37370, '82494L', 82482, '15056BL', 22411, 22803]
Overlap between GCNN and NeuMF: None


Comparison Summary
Overlap Between GCNN & NeuMF:
None. The two models produced completely different top-10 recommendations.

Overlap with Actual Purchases:
None of the top-10 recommended items from either GCNN or NeuMF appear in the actual purchases for customer 17850.

Inference:

Both models have distinct recommendation strategies. GCNN uses the graph structure of user-item interactions, while NeuMF relies on latent factor learning.

Neither model captured the customer’s actual preferences well in this instance — likely due to:

Sparse data for this customer.

Cold start effects.

Lack of side features (e.g., timestamps, categories).

# Step 3: Discussion of Results [20 points]

Discuss the performance of the GCNN model compared to the Feedforward NeuMF model. Provide insights on which model performs better and why, based on the evaluation metrics. Consider aspects like Precision@K, Recall@K, and F1 score.

Compare the recommended items for Customer 17850 generated by your model with those recommended by Neo4j. Are there similarities between the two sets of recommendations?


🧾 Recommendations for Customer ID 17850

GCNN  Top-10 StockCodes:  [22423, '85123A', 47566, 21212, 22720, 84879, '85099B', 22960, 23298, 22457]

NeuMF Top-10 StockCodes:  [85125, 37495, 23284, 23503, 21470, 22779, '35911B', 23418, 21220, '79063C']

Actual StockCodes (Purchased): ['84029E', 71053, 21730, '84406B', 22752, '85123A', '84029G', 22633, 22632, 20679, 21068, 21871, 82483, 21071, 82486, 37370, '82494L', 82482, '15056BL', 22411, 22803]

Overlap between GCNN and NeuMF: None




---------- Top 8 StockCodes bought by customer 17850 -----------

     CustomerID StockCode  Quantity
285       17850    82494L        12
2629      17850    85123A        12
2634      17850     71053        12
2978      17850     71053        12
2983      17850    85123A        12
3299      17850     71053        12
3301      17850     21068        12
3302      17850    84029G        12

------- Product Names of bought StockCodes ------

135    Mediven Sheer and Soft 15-20 mmHg Thigh w/ Lac...
162    Heavy Duty Handlebar Motorcycle Mount Holder K...
179           AARCO Enclosed Wall Mounted Bulletin Board
669    3 1/2"W x 20"D x 20"H Funston Craftsman Smooth...
967    Awkward Styles Shamrock Flag St. Patrick's Day...

------------ Recommendations for Customer 17850 -----------

 Overlap with Actual Purchases

✅ GCNN Recommendations:

Top-10:
[22423, '85123A', 47566, 21212, 22720, 84879, '85099B', 22960, 23298, 22457]


Match: '85123A' is present in actual purchases, and frequently bought (multiple times, Quantity = 12).


1 relevant hit, ranked 2nd — strong signal of relevance.


❌ NeuMF (Neo4j) Recommendations:

Top-10:
[85125, 37495, 23284, 23503, 21470, 22779, '35911B', 23418, 21220, '79063C']


No overlap with actual purchases.


🔄 Overlap Between GCNN and NeuMF
None: The two sets of recommendations have no StockCodes in common, indicating they’re likely using different latent structures or signals.


📊 Interpretation

✔ GCNN Performs Better
GCNN correctly identifies '85123A' as a relevant item.


'85123A' was purchased multiple times (at least twice in your sample).


This StockCode may represent a popular or personally relevant product for this customer — suggesting GCNN captures user preferences more effectively.


❌ NeuMF Misses the Target
NeuMF fails to recommend any item from the customer’s history.




